In [383]:
import imaplib
import email
from datetime import datetime, timedelta
import pytz

import yaml  #To load saved login credentials from a yaml file

In [384]:
with open("credentials.yml") as f:
    content = f.read()

In [385]:
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)

In [386]:

#Load the user name and passwd from yaml file
user, password = my_credentials["email_user"], my_credentials["email_password"]

In [387]:
#URL for IMAP connection
imap_url = 'imap.gmail.com'
# Connection with GMAIL using SSL
my_mail = imaplib.IMAP4_SSL(imap_url)


In [388]:
try:
    my_mail.login(user, password)
    print ("LOGIN SUCCESSFUL!!! ")
    # Select the Inbox to fetch messages
    my_mail.select('Inbox')
except imaplib.IMAP4.error:
    print ("LOGIN FAILED!!! ")

LOGIN SUCCESSFUL!!! 


In [389]:

#Define Key and Value for email search
#For other keys (criteria): https://gist.github.com/martinrusev/6121028#file-imap-search
key = 'FROM'
value = 'jrajpal2@illinois.edu'
_, data = my_mail.search(None, key, value)  #Search for emails with specific key and value

mail_id_list = data[0].split()  #IDs of all emails that we want to fetch 





In [390]:
msgs = [] # empty list to capture all messages
#Iterate through messages and extract data into the msgs list
for num in mail_id_list:
    typ, data = my_mail.fetch(num, '(RFC822)') #RFC822 returns whole message (BODY fetches just body)
    msgs.append(data)

In [391]:
#Now we have all messages, but with a lot of details
#Let us extract the right text and print on the screen

#In a multipart e-mail, email.message.Message.get_payload() returns a 
# list with one item for each part. The easiest way is to walk the message 
# and get the payload on each part:
# https://stackoverflow.com/questions/1463074/how-can-i-get-an-email-messages-text-content-using-python

# NOTE that a Message object consists of headers and payloads.

#create empty list storing course subject, number, and time of most recent notification
most_recent_notification = []
for response_part in msgs[-1]:
    if type(response_part) is tuple:
        my_msg=email.message_from_bytes((response_part[1]))
        for part in my_msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=False)
                print(body)
                #get day and time of email
                time = my_msg['Date']
                #find the word "Course" in the body
                course_index = body.find("Course:") + 8
                #get the course subject from course index to space
                course_subject = body[course_index:].split()[0]
                #get the course number from course index to space
                course_number = body[course_index:].split()[1]
                #convert time to datetime object
                print(time)
                time = datetime.strptime(time, '%a, %d %b %Y %H:%M:%S %z')
                most_recent_notification = (course_subject, course_number, time)
print(most_recent_notification)



________________________________
From: Course Explorer Notifications <techservices-no-reply@illinois.edu>
Sent: Wednesday, December 28, 2022 10:25:03 PM (UTC-06:00) Central Time (US=
 & Canada)
To: Rajpal, Jai Vishal <jrajpal2@illinois.edu>
Subject: Section Enrollment Status Update from Course Explorer


This is an automated message from the Course Explorer system notifying you =
that a seat has become available for the following section:

Term: Spring 2023
Course: KIN 104
CRN: 35023
Section: A6

This notification does not guarantee you the seat nor indicate that you are=
 eligible to enroll in that section.

View KIN 104<http://links.illinois.edu/f/a/aopbgPEHBQkm0wXAnhr4aw~~/AAMFlAA=
~/RgRlj5ufP0Q5aHR0cHM6Ly9jb3Vyc2VzLmlsbGlub2lzLmVkdS9zY2hlZHVsZS8yMDIzL3Nwc=
mluZy9LSU4vMTA0VwNzcGNCCmOinxatYw0pLp5SFWpyYWpwYWwyQGlsbGlub2lzLmVkdVgEAAAA=
DQ~~> in the Course Explorer.

Visit Student Self-Service<http://links.illinois.edu/f/a/7GOw8t6IUR3BJlz6GL=
Aiqg~~/AAMFlAA~/RgRlj5ufP0SIaHR0cHM6Ly93ZWJ

In [392]:
#check if the most recent notification is within the last 10 minutes
def check_event(most_recent_notification):
    utc=pytz.UTC
    most_recent_time = most_recent_notification[2]
    time_now = utc.localize(datetime.utcnow())
    return most_recent_time > time_now - timedelta(minutes=10)


In [393]:
check_event(most_recent_notification)

False